In [1]:
import pyspark

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName("Different-Caching").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/17 13:26:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/06/17 13:26:17 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/06/17 13:26:17 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 52813)
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/socketserver.py", line 318, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/socketserver.py", line 349, in process_request
    self.finish_request(request, client_address)
  File "/Library/Frameworks/Py

In [4]:
ls

ApplicationCodeOptimization.ipynb  ReduceByKey.ipynb
BroadcastJoins.ipynb               Repartiton-Coalesce.ipynb
Caching-Table.ipynb                Spark-Table.ipynb
Caching-performance.ipynb          SparkOptimizationcode.txt
Count.txt                          departments.csv
DataFrame.ipynb                    employees.csv
Different-caching.ipynb            output_data/
Distinct,Take.ipynb                output_data_parquet/
Filter.ipynb                       output_folder/
Group and Reduce By key.ipynb      spark-warehouse/
Joins.ipynb                        students.csv
Pyspark-Get.ipynb


In [9]:
empdf = spark.read.format("csv").option("inferSchema",True).option("header",True).load("employees.csv")

In [8]:
deptdf = spark.read.format("csv").option("inferSchema",True).option("header",True).load("departments.csv")

In [13]:
empdf.show(1)
deptdf.show(1)

+-----------+----------+---------+--------+------------+---------+--------+------+--------------+----------+-------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|  JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+----------+---------+--------+------------+---------+--------+------+--------------+----------+-------------+
|        198|    Donald| OConnell|DOCONNEL|650.507.9833|21-JUN-07|SH_CLERK|  2600|            - |       124|           50|
+-----------+----------+---------+--------+------------+---------+--------+------+--------------+----------+-------------+
only showing top 1 row

+-------------+---------------+----------+-----------+
|DEPARTMENT_ID|DEPARTMENT_NAME|MANAGER_ID|LOCATION_ID|
+-------------+---------------+----------+-----------+
|           10| Administration|       200|       1700|
+-------------+---------------+----------+-----------+
only showing top 1 row



In [18]:
cache_df = empdf.select("employee_id","first_name","last_name","salary").filter("salary>2600").cache()

In [19]:
empdf.select("employee_id","first_name","last_name","salary").filter("salary>2600").count()

39

In [ ]:
# here the logical plan changes
# because the above statement cache doen't have the information of the salary having greater tahn 10000

In [20]:
new_cache_df = empdf.select("employee_id","first_name","last_name","salary").filter("salary>10000")

In [21]:
new_cache_df.count()

7

In [22]:
# Now apply cache here

In [23]:
new_cache_df = empdf.select("employee_id","first_name","last_name","salary").filter("salary>10000").cache()

In [24]:
new_cache_df.count()

7

In [ ]:
# using RDD

In [25]:
rdd_cache = spark.sparkContext.textFile("count.txt").cache()

In [27]:
rdd_cache.collect()[2]

'hello world hello world'

In [28]:
rdd3 = rdd_cache.flatMap(lambda x: x.split(" "))

In [31]:
rdd4 = rdd3.map(lambda x:(x,1))

In [33]:
rdd5 = rdd4.reduceByKey(lambda x,y: x+y)

In [35]:
final_df = rdd5.toDF(["word","count"])

In [36]:
final_df.show()

+-------+-----+
|   word|count|
+-------+-----+
|     to|    1|
|   this|    2|
|       |    1|
|  hello|    2|
|  world|    2|
|welcome|    2|
|    the|    1|
|     is|    2|
|      a|    2|
+-------+-----+

